# 在 AlphaMed Notebook 调试环境中调试普通的模型

首先需要选择*我的 Notebook*打开 Notebook 调试环境，新建一个 Notebook 脚本文件。

![打开 Notebook 环境](res/2_1.png)

以下定义的 `ConvNet` 网络是一个简单的 CNN 网络，没有特殊之处。但是故意在代码中加入了 bug，以模拟调试过程中出错的情况。代码可以直接复制粘贴到 Notebook 中执行。

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=30, out_features=50)  # 故意设置了错误的 in_features 值
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

现在可以试一试是否能运行网络，期望会报错，且能够看到异常信息，以帮助调试。

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = ConvNet().to(device)
print(model)

X = torch.rand(1, 28, 28, device=device)
logits = model(X)

Using cpu device
ConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=30, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x320 and 30x50)

Notebook 环境正常显示异常信息，符合预期。接下来修正错误再尝试一下。

In [4]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=320, out_features=50)  # 修正了 in_features 值的错误
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [5]:
model = ConvNet().to(device)
print(model)

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

ConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Predicted class: tensor([7])


输出正常，符合预期。

将 `ConvNet` 的代码整理到 [res/cnn_net.py](res/cnn_net.py) 文件中，后续的示例中还会用到它。